In [1]:
#!/usr/bin/env python
# coding: utf-8
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torch.nn import TransformerEncoder, TransformerEncoderLayer

import torchtext
from torchtext import data, datasets
from torchtext.vocab import FastText

import os
import math
import MeCab

device="cuda:0" if torch.cuda.is_available() else "cpu"
SEED = 1
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
mecab = MeCab.Tagger("-Owakati")

# BLEU

from Util.selfbleu import CalcSelfBLEU
from Util.bleu import calc_all_bleu

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# HYPER PARAMETER
BATCH_SIZE=100
EMBED_SIZE=300
LSTM_HIDDEN_SIZE=300
AE_EPOCHS=100
REPORT_INTERVAL=5
ae_lr=1e-2

In [11]:
# LABEL NAME
data_source="wiki"
feature="only_ja_small"
feature="only_ja_sample"
# feature="all"

In [12]:
if data_source=="wiki" or data_source=="orphans":
    MAX_LEN=20 #paddingをバッチにかかわらず固定長にする
    TEXT = data.Field(sequential=True, lower=True, include_lengths=True, init_token="<SOS>",eos_token="<EOS>", fix_length=MAX_LEN, batch_first=True)
    train, val, test = data.TabularDataset.splits(
        path='./data/foruse', train="_".join(["train",data_source,feature])+'.tsv',
        validation="_".join(["eval",data_source,feature])+'.tsv',test="_".join(["test",data_source,feature])+'.tsv', 
        format='tsv',
        fields=[('Text', TEXT),])
    TEXT.build_vocab(train, vectors=FastText(language="ja"))
    train_iter, val_iter, test_iter = data.Iterator.splits((train, val, test), batch_sizes=(BATCH_SIZE, BATCH_SIZE, BATCH_SIZE),sort = False, device=torch.device(device))
    vocab_size=TEXT.vocab.vectors.size()[0]

In [13]:
share_conv=nn.Embedding(vocab_size, EMBED_SIZE).to(device)

In [14]:
class Encoder(nn.Module):
    def __init__(self,embed_size,n_hid):
        super().__init__()
        self.lstm = nn.LSTM(input_size = embed_size,
                            hidden_size = n_hid,
                            batch_first=True)
        
    def forward(self, x):
        """
        input: 
            x = (batch, max_len, embed_dim)
        output:
            x = (batch, hidden_size)
        """
        _, (h,c)=self.lstm(x) # h=(max_len, batch, n_hid)
        return (h,c)

In [15]:
class Decoder(nn.Module):
    def __init__(self,embed_size,n_hid,max_len,vocab_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size = embed_size,
                            hidden_size = n_hid,
                            batch_first=True)
        self.fc = nn.Linear(n_hid, vocab_size)
        self.lnorm=nn.LayerNorm(n_hid)
        self.max_len=max_len
        self.embedding=share_conv
        
    def forward(self, hidden, x, teacher):
        """
        input:
            x_emb=(batch, hidden_size, embedding_size)
        output:
            logits = (batch, max_len, vocab_size)
            sentence = (batch, max_len) : 中身はindex
        """
        logits = torch.tensor([]).to(device)
        sentence = torch.tensor([],dtype=torch.long).to(device)
        
        for i in range(self.max_len):
            if teacher or i==0:
                tmp = torch.unsqueeze(x[:,i,:],1) # tmp = (batch, 1, embed_dim)
            else:
                # word = (batch, 1, 1)
                tmp = self.embedding(word) # tmp = (batch, 1, embed_dim)
            x_input = tmp # x_input = (batch, 1, (embed_size + n_hid))
            out, hidden = self.lstm(x_input, hidden)
                # out = (batch, 1, n_hid)
                # hidden = ((batch, 1, n_hid),(batch, 1, n_hid))
            logit = self.fc(out) # logit = (batch, 1, vocab_size)
            word = torch.argmax(logit, dim=2) # word = (batch, 1)

            sentence = torch.cat([sentence, word],1)
            logits = torch.cat([logits,logit],1)   
        return logits, sentence

In [16]:
class AutoEncoder(nn.Module):
    def __init__(self,maxlen):
        super().__init__()
        self.maxlen=maxlen
        self.encoder=Encoder(embed_size=EMBED_SIZE,n_hid=LSTM_HIDDEN_SIZE)
        self.decoder=Decoder(embed_size=EMBED_SIZE,n_hid=LSTM_HIDDEN_SIZE,max_len=MAX_LEN,vocab_size=vocab_size)
        self.embedding=share_conv
        
    def forward(self, x): # x=(batch, max_len)
        x_emb=self.embedding(x)  # x_emb = (batch, maxlen, embed_dim)
        
        hidden = self.encoder(x_emb) # h,c = (batch, hidden_size)
        logits, sentence = self.decoder(hidden, x_emb, teacher=True)
        
        # loss
        criterion = nn.CrossEntropyLoss(ignore_index=1)
        loss = 0
        for i in range(self.maxlen-1):
            # <SOS>を除くためindexをずらす
            loss += criterion(torch.squeeze(logits[:,i,:]), torch.squeeze(x[:,i+1]))
        loss/=(self.maxlen-1)
        """
        # KL loss
        # 標準正規分布と(μ,σ^2)正規分布を仮定しているので以下の計算式になる
        # nn.klDivLossを使うと仮定が甘い
        # kl_loss = Σ0.5Σ(μ^2+exp(ln(σ^2))-ln(σ^2)-1)を使う
        kl_loss = torch.sum(0.5 * torch.sum((H_mean**2 + torch.exp(H_log_sigma_sq) - H_log_sigma_sq - 1),dim=1))
        loss += epoch/ae_epoch_number*kl_loss
        """
        
        return loss, sentence


In [17]:
def train():
    auto_encoder.train()
    epoch_loss = 0
    count=0
    for idx, batch in enumerate(train_iter):
        (x, x_l) = batch.Text
            # xには文章のID表現が、x_lにはxの単語数が入る
            # x=(batch, max_len)
        if len(x)!=BATCH_SIZE:break
        optimizer.zero_grad()
        loss, syn_sents=auto_encoder(x)
        with torch.autograd.detect_anomaly():
            loss.backward()
            optimizer.step()
        epoch_loss+=loss.item()
        count+=1
    sample_x=x[0][1:]
    source_sentence=' '.join([TEXT.vocab.itos[int(i)] for i in sample_x if i != 1])
    gen_sentence=' '.join([TEXT.vocab.itos[int(i)] for i in syn_sents[0] if i != 1])
    history_train.append(epoch_loss/count)
    if (epoch+1) % REPORT_INTERVAL==0:
        print("epoch: "+str(epoch+1)+'/'+str(AE_EPOCHS)+' ')
        print("training loss: "+str(history_train[epoch]))
#         print("kl_loss: "+str(kl_loss))
        print("source(train): "+str(source_sentence))
        print("result(train): "+str(gen_sentence))


In [18]:
'''
学習を始める
'''
print("start train...")
auto_encoder=AutoEncoder(maxlen=MAX_LEN)
auto_encoder.to(device)
optimizer = optim.Adam(auto_encoder.parameters(), lr=ae_lr)
history_train=[]
history_eval=[]

for epoch in range(AE_EPOCHS):
    train()

start train...
epoch: 5/100 
training loss: 3.064174175262451
source(train): 数 多く の 病院 が パリ に 設置 さ れ て いる <EOS>
result(train): この の の で で で ある 存在 て れ て いる <EOS> も <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
epoch: 10/100 
training loss: 1.3314015865325928
source(train): 表記 体系 は ほか の 諸 言語 と 比べ て 複雑 で ある <EOS>
result(train): この の は 現在 の 区別 言語 に 比べ て いる で ある <EOS> は <EOS> <EOS> <EOS> <EOS> <EOS>
epoch: 15/100 
training loss: 0.4283473491668701
source(train): 地理 学 誕生 の 地 は 古代 ギリシア で ある <EOS>
result(train): この 学 誕生 の 地 は 古代 ギリシア で ある <EOS> は を を 語 語 語 語 語 語
epoch: 20/100 
training loss: 0.1961958110332489
source(train): 言語 と 方言 の 区別 について 現在 なさ れる 説明 は 二つ で ある <EOS>
result(train): この と 方言 の 区別 について 現在 なさ れる 説明 は 二つ で ある <EOS> は が を を を
epoch: 25/100 
training loss: 0.11613862216472626
source(train): これら 化学 反応 が おこる 場 を 提供 し て いる の が 水 で ある <EOS>
result(train): これら 化学 反応 が おこる 場 を 提供 し て いる の が 水 で ある <EOS> も が を
epoch: 30/100 
training loss: 0.06710708886384964
source(train): この 他 教育 学部 に 設置 さ れ て い

## fasttext確認用

In [14]:
nihongo_id=0

In [125]:
nihongo_id+=1
norm_list=[]
word=TEXT.vocab.itos[nihongo_id]
nihongo_vector=share_conv.weight[nihongo_id]
for i, val in enumerate(share_conv.weight):
    diff=nihongo_vector-val
    norm=torch.norm(diff)
    norm_list.append((norm,i))
norm_list.sort()
print("「"+word+"」の似ている単語")
for i in range(1,11):
    val=norm_list[i][0]
    id_=norm_list[i][1]
    print(TEXT.vocab.itos[id_]+":"+str(val))

「方言」の似ている単語
方法:tensor(21.5570, device='cuda:0', grad_fn=<NormBackward0>)
ひとつ:tensor(21.7216, device='cuda:0', grad_fn=<NormBackward0>)
うち:tensor(21.8818, device='cuda:0', grad_fn=<NormBackward0>)
独自:tensor(21.9748, device='cuda:0', grad_fn=<NormBackward0>)
県庁:tensor(22.0037, device='cuda:0', grad_fn=<NormBackward0>)
品詞:tensor(22.0942, device='cuda:0', grad_fn=<NormBackward0>)
ゆれ:tensor(22.1198, device='cuda:0', grad_fn=<NormBackward0>)
日干し:tensor(22.1313, device='cuda:0', grad_fn=<NormBackward0>)
にくい:tensor(22.2012, device='cuda:0', grad_fn=<NormBackward0>)
生活:tensor(22.2619, device='cuda:0', grad_fn=<NormBackward0>)
